In [22]:
!pip install transformers

In [2]:
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 MB 121.6 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 101.8 MB/s  0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12━━━━━━━ 0/4 [sympy]
  Attempting uninstall: torch━━━━━━━━━━━━━━━━━━━ 0/4 [sympy]
    Found existing installation: torch 2.1.2 0/4 [sympy]
    Uninstalling torch-2.1.2:━━━━━━━━━━━━━━━ 0/4 [sympy]
      Successfully uninstalled torch-2.1.2━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [torch]
  Attempting uninstall: torchvision0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [torch]
    Found existing installation: torchvision 0.16.2━━━━━━━━━━━ 1/4 [torch]
    Uninstalling torchvision-0.16.2:━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [torch]
      Successfully uninstalled torchvision-0.16.2━━━━━━━

In [1]:
import torch
print(torch.__version__)

from transformers import AutoTokenizer
print("Transformers imported successfully!")

2.6.0+cpu
Transformers imported successfully!


In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [3]:
ids = tokenizer("It was a dark and stormy",return_tensors = "pt").input_ids
ids

tensor([[1026,  373,  257, 3223,  290, 6388,   88]])

In [4]:
ids[0]

tensor([1026,  373,  257, 3223,  290, 6388,   88])

In [5]:
ids[0][1]

tensor(373)

In [4]:
for t in ids[0]:
    print(t,"\t:",tokenizer.decode(t))


tensor(1026) 	: It
tensor(373) 	:  was
tensor(257) 	:  a
tensor(3223) 	:  dark
tensor(290) 	:  and
tensor(6388) 	:  storm
tensor(88) 	: y


In [6]:
from transformers import AutoModelForCausalLM
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")  #,pad_token_id = tokenizer.eos_token_id)

In [7]:
output = gpt2(ids)
output.logits.shape
# output.logits.shape: (batch_size, sequence_length, vocab_size)

torch.Size([1, 7, 50257])

In [8]:
output

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -37.0890,  -36.4551,  -40.3287,  ...,  -45.2598,  -43.0251,
           -37.6606],
         [-120.1124, -119.2538, -125.9656,  ..., -126.5557, -123.9601,
          -123.0143],
         [-115.5612, -113.6851, -118.5185,  ..., -122.3478, -118.8636,
          -115.7403],
         ...,
         [ -95.5841,  -92.1287,  -98.0233,  ..., -100.4968,  -96.3200,
           -96.0847],
         [ -63.8667,  -62.5591,  -67.8027,  ...,  -77.6469,  -70.6219,
           -65.7388],
         [-114.5009, -112.2047, -120.4313,  ..., -124.7593, -121.1788,
          -116.3877]]], grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-1.4202e+00,  1.6791e+00,  9.8370e-01,  ..., -1.1423e+00,
           -5.7307e-01,  1.1510e+00],
          [-1.7495e+00,  2.2843e+00,  1.8154e+00,  ..., -1.5640e+00,
           -2.6205e+00,  2.4664e+00],
          [-2.3145e+00,  2.7101e+00,  1.5073e+00,  ..., -5.7809e-01,
           -1.9292e+00,  2.2634e+00],
   

In [11]:
final_logits= output.logits[0][6]   #gpt2(ids).logits[0][-1]
final_logits.argmax()

tensor(1755)

In [12]:
tokenizer.decode(final_logits.argmax())

' night'

In [14]:
import torch
top10_logits = torch.topk(final_logits,10)
for index in top10_logits.indices:
    print(tokenizer.decode(index))



 night
 day
 evening
 morning
 afternoon
 summer
 time
 winter
 weekend
,


In [16]:
top10 = torch.topk(final_logits.softmax(dim=0),10)
for value,index in zip(top10.values,top10.indices):
    print(f"{tokenizer.decode(index):<10} {value.item():.2%}")

 night     46.18%
 day       23.46%
 evening   5.87%
 morning   4.42%
 afternoon 4.11%
 summer    1.34%
 time      1.33%
 winter    1.22%
 weekend   0.39%
,          0.38%


In [19]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [21]:
output_ids = gpt2.generate(ids,max_new_tokens=20,pad_token_id = tokenizer.pad_token_id)
decoded_text = tokenizer.decode(output_ids[0])

print("input ids:",ids[0])
print("output ids:",output_ids[0])
print(f"generated text:{decoded_text}")

input ids: tensor([1026,  373,  257, 3223,  290, 6388,   88])
output ids: tensor([ 1026,   373,   257,  3223,   290,  6388,    88,  1755,    13,   383,
         2344,   373, 19280,    11,   290,   262, 15114,   547,  7463,    13,
          383,  2344,   373, 19280,    11,   290,   262])
generated text:It was a dark and stormy night. The wind was blowing, and the clouds were falling. The wind was blowing, and the


In [13]:
output

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -37.0890,  -36.4551,  -40.3287,  ...,  -45.2598,  -43.0251,
           -37.6606],
         [-120.1124, -119.2538, -125.9656,  ..., -126.5557, -123.9601,
          -123.0143],
         [-115.5612, -113.6851, -118.5185,  ..., -122.3478, -118.8636,
          -115.7403],
         ...,
         [ -95.5841,  -92.1287,  -98.0233,  ..., -100.4968,  -96.3200,
           -96.0847],
         [ -63.8667,  -62.5591,  -67.8027,  ...,  -77.6469,  -70.6219,
           -65.7388],
         [-114.5009, -112.2047, -120.4313,  ..., -124.7593, -121.1788,
          -116.3877]]], grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-1.4202e+00,  1.6791e+00,  9.8370e-01,  ..., -1.1423e+00,
           -5.7307e-01,  1.1510e+00],
          [-1.7495e+00,  2.2843e+00,  1.8154e+00,  ..., -1.5640e+00,
           -2.6205e+00,  2.4664e+00],
          [-2.3145e+00,  2.7101e+00,  1.5073e+00,  ..., -5.7809e-01,
           -1.9292e+00,  2.2634e+00],
   